# Код для получения списка профилей соотвествующих поисковым запросам в LinkedIn

In [ ]:
import time
import configparser
import random
import re
import os.path

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

**Загружаем конфиг**

In [ ]:
# путь к файлу расширения для Chrome "Доступ к LinkedIn"
EXTENSION_PATH = '1.5_0.crx'

# конфиг
CFG_FILE = 'parser.ini'

DATA_PATH = '../datasets/'

"""
файл конфигурации необходимо предварительно создать,
формат файла parser.ini:
[LINKEDIN]
USER_LOGIN = эл_почта_без_кавычек
USER_PASSWORD = пароль
""";

# загружаем данные из конфига
conf = configparser.ConfigParser()
try:
    conf.read(CFG_FILE)
    USER_LOGIN = conf['LINKEDIN']['USER_LOGIN']
    USER_PASSWORD = conf['LINKEDIN']['USER_PASSWORD']
except:
    print(f'Не удалось прочитать файл конфигурации: {CFG_FILE}')

**Общие процедуры и функции**

In [ ]:
# формируем запрос на поиск людей по ключевым словам
def search_people_url(keywords, tags, page_num=1):
    """
    Функция на вход получает ключевые слова,
    список тем публикаций для поиска и номер страницы.
    Возвращает url для запроса страницы.
    """
    # преобразуем теги
    tags_str = str(tags).replace(" ", "").replace("'", '"')
    
    # формируем строку запроса
    search_url = 'https://www.linkedin.com/search/results/people/'
    search_url += f'?keywords={keywords}'
    search_url += '&origin=FACETED_SEARCH'
    search_url += f'&page={page_num}'
    search_url += '&profileLanguage=["ru"]'
    # темы публикаций (хештеги)
    search_url += f'&talksAbout={tags_str}'
    
    return search_url

In [ ]:
# получаем список профилей на странице
def get_profiles(driver):
    """
    Функция получает драйвер открытой страницы,
    ищет ссылки на доступные профили пользователей и возвращает
    список id пользователей.
    """
    # список найденных профилей
    profiles = []

    # ищем на странице ссылки на профили
    finded_profiles = driver.find_elements(
        By.CSS_SELECTOR, "span.entity-result__title-text a.app-aware-link"
    )
    for profile in finded_profiles:
        # получаем url на профиль пользователя
        url = profile.get_attribute("href")
        # если url ссылается на доступный профиль
        if 'linkedin.com/in' in url:
            # оставляем только id профиля
            profile_id = url.split('?')[0].split('/in/')[1]
            # добавляем id в список
            profiles.append(profile_id)

    # избавляемся от дублей, если вдруг появятся
    profiles = list(set(profiles))
    return profiles

**Создаем и запускаем браузер**

In [ ]:
# подключаем расширение к драйверу
options = webdriver.ChromeOptions()
options.add_extension(EXTENSION_PATH)

# меняем стратегию - ждать, пока свойство document.readyState примет значение interactive
options.page_load_strategy = 'eager'

# запускаем Chrome с расширением
driver = webdriver.Chrome(options=options)

**Вход в LinkedIn**

In [ ]:
# открываем страницу входа linkedIn, необходимо отключить двухфакторную аутонтификацию
driver.get("https://linkedin.com/uas/login")

# ожидаем загрузку страницы
time.sleep(3)

# поле ввода имени пользователя
username = driver.find_element(By.ID, "username")
# вводим свой Email
username.send_keys(USER_LOGIN)

# поле ввода пароля
pword = driver.find_element(By.ID, "password")
# вводим пароль
pword.send_keys(USER_PASSWORD)

# нажимаем кнопку Войти
driver.find_element(By.XPATH, "//button[@type='submit']").click()

**Параметры парсинга**

In [ ]:
# параметры поисковых запросов, теги, темы публикаций

#KEYWORDS = 'разработка по'
#TAGS = ['softwaredevelopment', 'webdevelopment', 'startup', 'it', 'design']
#CSV_FILE_NAME = os.path.join(DATA_PATH, 'profiles_id_1.csv')

#KEYWORDS = 'devops'
#TAGS = ['devops', 'aws', 'python', 'cloud', 'kubernetes']
#CSV_FILE_NAME = os.path.join(DATA_PATH, 'profiles_id_2.csv')

#KEYWORDS = 'data science'
#TAGS = ['datascience', 'machinelearning', 'ai', 'artificialintelligence', 'dataanalytics']
#CSV_FILE_NAME = os.path.join(DATA_PATH, 'profiles_id_3.csv')

#KEYWORDS = 'project management'
#TAGS = ['projectmanagement', 'business', 'agile', 'scrum', 'it']
#CSV_FILE_NAME = os.path.join(DATA_PATH, 'profiles_id_4.csv')

KEYWORDS = 'design ui ux'
TAGS = ['design', 'webdesign', 'ux', 'ui', 'uxdesign', 'uidesign']
CSV_FILE_NAME = os.path.join(DATA_PATH, 'profiles_id_5.csv')

**Собираем ID пользователей**

In [ ]:
# число страниц для парсинга, в бесплатном аккаунте доступно не более 100
NUM_PAGES = 100

# пустой датафрейм для id пользователей
df = pd.DataFrame(columns=['id'])

for page_num in range(1, NUM_PAGES+1):
    
    # выводим номер страницы, в случае сбоя можно будет начать новый парсинг с нее
    print(page_num, end='  ')
    
    # формируем url запроса
    people_url = search_people_url(KEYWORDS, TAGS, page_num=page_num)
    
    # запрашиваем и открываем страницу
    driver.get(people_url)
    
    # получаем и добавляем список найденных id профилей на странице
    profiles_id = get_profiles(driver)
    
    # добавляем данные в датафрейм
    df = pd.concat(
        [df, pd.DataFrame({'id': profiles_id})]
    ).reset_index(drop=True)
    
    # сохраняем в CSV
    df.to_csv(CSV_FILE_NAME)

    time.sleep(random.uniform(3, 5))

In [ ]:
# закрываем браузер
driver.quit()

**Результат**

In [ ]:
# профили
df.info()